In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.dicts_and_lists import *
from helperToolz.guzinski import * 
from helperToolz.mirmazloumi import *
from pympler import asizeof
from other_repos.pyTSEB.pyTSEB import meteo_utils
from other_repos.pyTSEB.pyTSEB import resistances
from other_repos.pyTSEB.pyTSEB import net_radiation
from other_repos.pyTSEB.pyTSEB import clumping_index 
from other_repos.pyTSEB.pyTSEB import TSEB
import rasterio

In [2]:
base_path = '/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/sharpened/'

folders = ['S2only', 'all_preds']

comps = ['maxLST', 'minVZA']

days = [f'July_{i:02d}' for i in range(1,10,1)]

masks = ['S2Masked_withLSTmask', 'S2Masked_withoutLSTmask', 'S2notMasked_withLSTmask', 'S2notMasked_withoutLSTmask']

file_dict = {}

for folder in folders:
    for comp in comps:
        for day in days:
            for mask in masks:
                key = (folder, comp, day, mask)
                file_list = [
                    file for file in getFilelist(base_path + folder, '.tif', deep=True)
                    if comp in file and day in file and mask in file and 'resid' not in file
                ]
                file_dict[key] = file_list

In [3]:
# set output path
tempOut = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/44/'

# set the year, month and day to estimate evapotranspiration for
year = 2019
month = 'July'
day = 9
comp = 'minVZA'

# set the parameters from sharpener
mvwin = 0
cv = 15
regrat = 25

s2_masked = False
lst_masked = False

if s2_masked:
    s2Mask = 'S2Masked'
else:
    s2Mask = 'S2notMasked'

if lst_masked:
    lstMask = 'withLSTmask'
else:
    lstMask = 'withoutLSTmask'


LST_files = file_dict[('all_preds', comp, f'{month}_{day:02d}', f'{s2Mask}_{lstMask}')]

In [4]:
# path to era5 raw data
era5_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/'
ssrd_mean_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/ssrd_mean_calc/'

# the LST acquisition time should determine which sharpened LST files are associatedto be processed (as they are associated with it)
LST_acq_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/Acq_time/int_format/{year}/Daily_AcqTime_{comp}_{year}_{month}.tif' # epsg 4326

# the VZA at the time of LST acquisition is need
VZA_at_acq_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/VZA/comp/{comp}/{year}/Daily_VZA_{comp}_{year}_{month}.tif' # epsg 4326

# the DEM, SLOPE, ASPECT, LAT, LON will be used to sharpen some of the era5 variables (the the resolution of the DEM)
dem_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/DEM_GER_LST_WARP.tif' # epsg 4326
slope_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/SLOPE_GER_LST_WARP.tif' # epsg 4326
aspect_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/ASPECT_GER_LST_WARP.tif' # epsg 4326
lat_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LAT_GER_LST_WARP.tif' # epsg 4326
lon_path = '/data/Aldhani/eoagritwin/et/Auxiliary/DEM/reprojected/LON_GER_LST_WARP.tif' # epsg 4326

# the geopotential is needed for the sharpening as well
geopot_path = '/data/Aldhani/eoagritwin/et/Auxiliary/ERA5/tiff/low_res/geopotential/geopotential_low_res.tif' # epsg 4326

# sharpened LST
LST_file = LST_files[0]
# for NDVI calculation (estimating LAI and others) and warping to S2 resolution, we use the S2 composite used for sharpening

# S2_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/tempDump/{LST_file.split(f'/{comp}')[0].split('/')[-1]}/HIGHRES_{comp}_{year}_{month}_{day:02d}_watermask.tif'
S2_file = f'/data/Aldhani/eoagritwin/et/Sentinel3/LST/LST_values/tempDump/{LST_file.split(f'/{comp}')[0].split('/')[-1]}/S2_20190705.vrt'
# find era5 file that matches the month of LST observation
valid_variables = sorted(list(dict.fromkeys(file.split('/')[-2] for file in getFilelist(era5_path, '.grib', deep=True) \
                                   if not any(var in file for var in ['geopotential', 'total_column_water_vapour']))))

# get a list for those era5 files that match the year and month of the provided LST acquisition file
era5_path_list = find_grib_file(getFilelist(era5_path, '.grib', deep=True), LST_acq_file)
era5_path_list = [path for path in era5_path_list if any(variable in path for variable in valid_variables)] # era5 are epsg 4326 and still will be after warping to doy
temp_pressure_checker(era5_path_list)

temperature will be processed before surface pressure - continue


In [5]:
# warp datasets needed for calculations to the spatial extent of the sharpened LST
LST_acq_spatial_sub = warp_raster_to_reference(source_path=LST_acq_file, reference_path=S2_file, output_path='MEM', resampling='near', keepRes=False)
VZA_at_acq_file_sub = warp_raster_to_reference(source_path=VZA_at_acq_file, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=False)
dem_sub = warp_raster_to_reference(source_path=dem_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=False)
slope_sub  = warp_raster_to_reference(source_path=slope_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=False)
aspect_sub = warp_raster_to_reference(source_path=aspect_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=False)
lat_sub = warp_raster_to_reference(source_path=lat_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=False)
lon_sub = warp_raster_to_reference(source_path=lon_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=False)
geopot_sub = warp_raster_to_reference(source_path=geopot_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=False)

/data/Aldhani/users/potzschf/conda/envs/cds_era5/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [ ]:
# # warp datasets needed for calculations to the spatial extent of the sharpened LST
# LST_acq_spatial_sub = warp_raster_to_reference(source_path=LST_acq_file, reference_path=S2_file, output_path='MEM', resampling='near', keepRes=True)
# VZA_at_acq_file_sub = warp_raster_to_reference(source_path=VZA_at_acq_file, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)
# dem_sub = warp_raster_to_reference(source_path=dem_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)
# slope_sub  = warp_raster_to_reference(source_path=slope_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)
# aspect_sub = warp_raster_to_reference(source_path=aspect_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)
# lat_sub = warp_raster_to_reference(source_path=lat_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)
# lon_sub = warp_raster_to_reference(source_path=lon_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)
# geopot_sub = warp_raster_to_reference(source_path=geopot_path, reference_path=S2_file, output_path='MEM', resampling='bilinear', keepRes=True)

In [6]:
# load the era5 variable into cache at LST resolution and read-in the modelled times (one time step per band)
for path in era5_path_list:
    print(f'processing {path}')
    # check if DEM sharpener needs to be applied
    if '100m_u_component_of_wind' in path:
        # do the warping without sharpening
        wind100_u = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, lst_acq_file=LST_acq_spatial_sub, doy=day)

    elif '100m_v_component_of_wind' in path:
               # do the warping without sharpening
        wind100_v = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, lst_acq_file=LST_acq_spatial_sub, doy=day)

    # elif 'geopotential' in path:
    #     # do the warping without sharpening
    #     geopot = get_warped_ERA5_at_doy(path_to_era_grib=path, lst_acq_file=LST_acq_file, doy=day)

    elif 'downward' in path: # terrain correction included
        ssrd, szenith, sazimuth, ssrd_nc = get_ssrdsc_warped_and_corrected_at_doy(path_to_ssrdsc_grib=path, reference_path=LST_acq_spatial_sub, 
                                                                         lst_acq_file=LST_acq_spatial_sub, doy=day, 
                                                                         slope_path=slope_sub,
                                                                         aspect_path=aspect_sub,
                                                                         dem_path=dem_sub,
                                                                         lat_path=lat_sub,
                                                                         lon_path=lon_sub, nodat=0)

    elif '2m_temperature' in path: # DEM and adiabatic sharpening, following Guzinski 2021
        air_temp = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, 
                                          lst_acq_file=LST_acq_spatial_sub, doy=day,
                                          sharp_blendheight=100,
                                          sharp_DEM=dem_sub,
                                          sharp_geopot=geopot_sub,
                                          sharp_rate=STANDARD_ADIABAT,
                                          sharpener='adiabatic', nodat=0)

    elif '2m_dewpoint_temperature' in path: # DEM and adiabatic sharpening, following Guzinski 2021
        dew_temp = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, 
                                          lst_acq_file=LST_acq_spatial_sub, doy=day,
                                          sharp_blendheight=100,
                                          sharp_DEM=dem_sub,
                                          sharp_geopot=geopot_sub,
                                          sharp_rate=MOIST_ADIABAT,
                                          sharpener='adiabatic', nodat=0)

    else: 
        # do warping with DEM sharpening only
        # sanity check
        if not 'surface_pressure' in path:
            raise ValueError('There is and unattended ERA5 variable in the loop - CHECK!!!!')
        else:
            sp = get_warped_ERA5_at_doy(path_to_era_grib=path, reference_path=LST_acq_spatial_sub, 
                                        lst_acq_file=LST_acq_spatial_sub, doy=day,
                                        sharp_DEM=dem_sub,
                                        sharp_blendheight=100,
                                        sharp_geopot=geopot_sub,
                                        sharp_temp=air_temp,
                                        sharpener='barometric', nodat=0)

# load the mean ssrd
ds = gdal.Open(f'{ssrd_mean_path}surface_solar_radiation_downward_clear_sky_{year}_{int(MONTH_TO_02D[month])}')
ssrd_mean = ds.GetRasterBand(day).ReadAsArray() / 3600

processing /data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/100m_u_component_of_wind/100m_u_component_of_wind_2019_7.grib
processing /data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/100m_v_component_of_wind/100m_v_component_of_wind_2019_7.grib
processing /data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/2m_dewpoint_temperature/2m_dewpoint_temperature_2019_7.grib
Adiabatic Sharpener will be applied
processing /data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/2m_temperature/2m_temperature_2019_7.grib
Adiabatic Sharpener will be applied
processing /data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/surface_pressure/surface_pressure_2019_7.grib
Barometric Sharpener will be applied


/home/potzschf/repos/helperToolz/guzinski.py:636: RuntimeWarning: divide by zero encountered in divide
  ratio = (era5_corrected_Temp - STANDARD_ADIABAT * delta_z) / era5_corrected_Temp
/home/potzschf/repos/helperToolz/guzinski.py:360: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(block, axis = 2)


processing /data/Aldhani/eoagritwin/et/Auxiliary/ERA5/grib/surface_solar_radiation_downward_clear_sky/surface_solar_radiation_downward_clear_sky_2019_7.grib


/home/potzschf/repos/helperToolz/guzinski.py:525: RuntimeWarning: All-NaN slice encountered
  ssrd_watt = np.nanmax(block, axis = 2)


In [ ]:
# npTOdisk(ssrd_mean, LST_acq_spatial_sub, f'{tempOut}SSRD_low.tif', bands = 1)
# npTOdisk(ssrd, LST_acq_spatial_sub, f'{tempOut}SSRD_low.tif', bands = 1)
# npTOdisk(air_temp, LST_acq_spatial_sub, f'{tempOut}TEMP_low.tif', bands = 1)
# npTOdisk(dew_temp, LST_acq_spatial_sub, f'{tempOut}DEW_low.tif', bands = 1)
# npTOdisk(sp, LST_acq_spatial_sub, f'{tempOut}SP_low.tif', bands = 1)
# npTOdisk(szenith, LST_acq_spatial_sub, f'{tempOut}ZEN_low.tif', bands = 1)
# npTOdisk(sazimuth, LST_acq_spatial_sub, f'{tempOut}AZI_low.tif', bands = 1)
# npTOdisk(wind100_u, LST_acq_spatial_sub, f'{tempOut}WSPEED_U_low.tif', bands = 1)
# npTOdisk(wind100_v, LST_acq_spatial_sub, f'{tempOut}WSPEED_V_low.tif', bands = 1)

In [7]:
# bring all variables down to 20m
wind100_u_20 = wind100_u
wind100_v_20 = wind100_v
ssrd_20 = ssrd
ssrd_mean_20 = warp_np_to_reference(ssrd_mean, f'{ssrd_mean_path}surface_solar_radiation_downward_clear_sky_{year}_{int(MONTH_TO_02D[month])}', LST_file) # check this too!!!!!!
air_temp_20 = air_temp
dew_temp_20 = dew_temp
sp_20 = sp
szenith_20 = szenith
sazimuth_20 = sazimuth

# calculate windspeed
wind_speed_20 = calc_wind_speed(wind100_u_20, wind100_v_20) # check if this really works

# load vza

vza_20 = VZA_at_acq_file_sub.GetRasterBand(day).ReadAsArray()
# load sharpened LST
lst_ds = gdal.Open(LST_file)
lst_20 =lst_ds.GetRasterBand(1).ReadAsArray()


In [ ]:
# # bring all variables down to 20m
# wind100_u_20 = warp_np_to_reference(wind100_u, LST_acq_spatial_sub, LST_file)
# wind100_v_20 = warp_np_to_reference(wind100_v, LST_acq_spatial_sub, LST_file)
# ssrd_20 = warp_np_to_reference(ssrd, LST_acq_spatial_sub, LST_file) # check this too!!!!!!
# ssrd_mean_20 = warp_np_to_reference(ssrd_mean, f'{ssrd_mean_path}surface_solar_radiation_downward_clear_sky_{year}_{int(MONTH_TO_02D[month])}', LST_file) # check this too!!!!!!
# air_temp_20 = warp_np_to_reference(air_temp, LST_acq_spatial_sub, LST_file)
# dew_temp_20 = warp_np_to_reference(dew_temp, LST_acq_spatial_sub, LST_file)
# sp_20 = warp_np_to_reference(sp, LST_acq_spatial_sub, LST_file)
# szenith_20 = warp_np_to_reference(szenith, LST_acq_spatial_sub, LST_file)
# sazimuth_20 = warp_np_to_reference(sazimuth, LST_acq_spatial_sub, LST_file)

# # calculate windspeed
# wind_speed_20 = calc_wind_speed(wind100_u_20, wind100_v_20) # check if this really works

# # load vza
# vza_ds = warp_raster_to_reference(VZA_at_acq_file_sub, LST_file, output_path='MEM', resampling='bilinear')
# vza_20 = vza_ds.GetRasterBand(day).ReadAsArray()
# # load sharpened LST
# lst_ds = gdal.Open(LST_file)
# lst_20 =lst_ds.GetRasterBand(1).ReadAsArray()

# del wind100_u, wind100_v, ssrd, air_temp, dew_temp, sp, wind100_u_20, wind100_v_20, szenith, sazimuth, ssrd_mean

In [ ]:
# npTOdisk(ssrd_mean_20, LST_file, f'{tempOut}SSRD_mean.tif', bands = 1)
# npTOdisk(ssrd_20, LST_file, f'{tempOut}SSRD.tif', bands = 1)
# npTOdisk(air_temp_20, LST_file, f'{tempOut}TEMP.tif', bands = 1)
# npTOdisk(dew_temp_20, LST_file, f'{tempOut}DEW.tif', bands = 1)
# npTOdisk(sp_20, LST_file, f'{tempOut}SP.tif', bands = 1)
# npTOdisk(szenith_20, LST_file, f'{tempOut}ZEN.tif', bands = 1)
# npTOdisk(sazimuth_20, LST_file, f'{tempOut}AZI.tif', bands = 1)
# npTOdisk(wind_speed_20, LST_file, f'{tempOut}WSPEED.tif', bands = 1)
# npTOdisk(lst_20, LST_file, f'{tempOut}LST.tif', bands = 1)
# npTOdisk(vza_20, LST_file, f'{tempOut}VZA.tif', bands = 1)

In [8]:
condition = (air_temp_20 > 0) & (dew_temp_20 > 0)  & (sp_20 > 0) & (szenith_20 > 0) & (sazimuth_20 > 0) & (wind_speed_20 > 0) & (lst_20 > 0) & (vza_20 > 0)
ssrd_20[~condition] = np.nan
ssrd_mean_20[~condition] = np.nan
air_temp_20[~condition] = np.nan
dew_temp_20[~condition] = np.nan
sp_20[~condition] = np.nan
szenith_20[~condition] = np.nan
sazimuth_20[~condition] = np.nan
wind_speed_20[~condition] = np.nan
# lst_20 = np.ma.masked_where(~condition, lst_20)
# vza_20 = np.ma.masked_where(~condition, vza_20)
lst_20[~condition] = np.nan
vza_20[~condition] = np.nan

In [ ]:
# npTOdisk(ssrd_mean_20, LST_file, f'{tempOut}SSRD_mean_mask.tif', bands = 1)
# npTOdisk(ssrd_20, LST_file, f'{tempOut}SSRD_mask.tif', bands = 1)
# npTOdisk(air_temp_20, LST_file, f'{tempOut}TEMP_mask.tif', bands = 1)
# npTOdisk(dew_temp_20, LST_file, f'{tempOut}DEW_mask.tif', bands = 1)
# npTOdisk(sp_20, LST_file, f'{tempOut}SP_mask.tif', bands = 1)
# npTOdisk(szenith_20, LST_file, f'{tempOut}ZEN_mask.tif', bands = 1)
# npTOdisk(sazimuth_20, LST_file, f'{tempOut}AZI_mask.tif', bands = 1)
# npTOdisk(wind_speed_20, LST_file, f'{tempOut}WSPEED_mask.tif', bands = 1)
# npTOdisk(lst_20, LST_file, f'{tempOut}LST_mask.tif', bands = 1)
# npTOdisk(vza_20, LST_file, f'{tempOut}VZA_mask.tif', bands = 1)

In [9]:
# calculate the NDVI from the S2 composite (following formula from force --> bandnames: (NIR - RED) / (NIR + RED))
S2_ds = gdal.Open(S2_file)
for idx, bname in enumerate(getBandNames(S2_file)):
    if bname == 'RED':
        red = S2_ds.GetRasterBand(1 + idx).ReadAsArray()
    elif bname == 'NIR':
        nir = S2_ds.GetRasterBand(1 + idx).ReadAsArray()
    else:
        continue
ndvi_20 = (nir - red) / (nir + red)
ndvi_20_ma = np.ma.masked_invalid(ndvi_20)
ndvi_20_ma = np.ma.masked_where(ndvi_20_ma < 0, ndvi_20_ma)
LAI_np = 0.57*np.exp(2.33*ndvi_20)
LAI_pos = np.ma.masked_where(LAI_np < 0, LAI_np)

# estimate canopy height from estimated LAI
hc = hc_from_lai(LAI_pos, hc_max = 1.2, lai_max = np.nanmax(LAI_np), hc_min=0)

# estimate long wave irradiance
ea = meteo_utils.calc_vapor_pressure(T_K=dew_temp_20)
L_dn = calc_longwave_irradiance(ea = ea, t_a_k = air_temp_20, p = sp_20, z_T = 100, h_C = hc) # ## does that make sense with the 100m!!!!!!!!!!!!!!!!!!!
d_0_0 = resistances.calc_d_0(h_C=hc)
z_0 = resistances.calc_z_0M(h_C=hc)

In [10]:
# calculate shortwave radiation of soil and canopy
difvis, difnir, fvis, fnir = net_radiation.calc_difuse_ratio(S_dn = ssrd_20, sza = np.mean(szenith_20))

skyl = difvis * fvis + difnir * fnir
S_dn_dir = ssrd_20 * (1.0 - skyl)
S_dn_dif = ssrd_20 * skyl

# Leaf spectral properties:{rho_vis_C: visible reflectance, tau_vis_C: visible transmittance, rho_nir_C: NIR reflectance, tau_nir_C: NIR transmittance}
rho_vis_C=np.full(LAI_pos.shape, 0.05, np.float32)
tau_vis_C=np.full(LAI_pos.shape, 0.08, np.float32)
rho_nir_C=np.full(LAI_pos.shape, 0.32, np.float32)
tau_nir_C=np.full(LAI_pos.shape, 0.33, np.float32) 

# Soil spectral properties:{rho_vis_S: visible reflectance, rho_nir_S: NIR reflectance}
rho_vis_S=np.full(LAI_pos.shape, 0.07, np.float32)
rho_nir_S=np.full(LAI_pos.shape, 0.25, np.float32)

# F = local LAI
F = LAI_pos / 1
# calculate clumping index
Omega0 = clumping_index.calc_omega0_Kustas(LAI = LAI_np, f_C = 1, x_LAD=1)
Omega = clumping_index.calc_omega_Kustas(Omega0, np.mean(szenith_20))
LAI_eff = F * Omega

Sn_C, Sn_S = net_radiation.calc_Sn_Campbell(lai = LAI_pos, sza = np.mean(szenith_20), S_dn_dir = S_dn_dir, S_dn_dif = S_dn_dif, fvis = fvis,
                                    fnir = fnir, rho_leaf_vis = rho_vis_C, tau_leaf_vis = tau_vis_C, rho_leaf_nir = rho_nir_C, 
                                    tau_leaf_nir = tau_nir_C, rsoilv = rho_vis_S, rsoiln = rho_nir_S, x_LAD=1, LAI_eff=LAI_eff)

# calculate other roughness stuff
z_0M, d = resistances.calc_roughness(LAI=np.nanmean(LAI_pos), h_C=hc, w_C=1, landcover=11, f_c=None)
z_0M_array = np.full(LAI_pos.shape, z_0M)
d_array = np.full(LAI_pos.shape, d)
fg = calc_fg_gutman(ndvi = ndvi_20_ma, ndvi_min = np.nanmin(ndvi_20), ndvi_max = np.nanmax(ndvi_20))

In [11]:
emis_C = 0.98
emis_S = 0.95
h_C = hc 
z_u = 100
z_T = 100

In [12]:
output = TSEB.TSEB_PT(lst_20, vza_20, air_temp_20, wind_speed_20, ea, sp_20, Sn_C, Sn_S, L_dn, LAI_pos, h_C, emis_C, emis_S, 
                      z_0M, z_0M_array, z_u, z_T, resistance_form=None, calcG_params=None, const_L=None, 
                      kB=0.0, massman_profile=None, verbose=True)

len(output)
ld = output[6]/ssrd_20
heat_latent_scaled = ssrd_mean_20 * ld
et_daily_p = TSEB.met.flux_2_evaporation(heat_latent_scaled, t_k=air_temp_20, time_domain=24)

Iteration: 0, non-converged pixels: 1053694, max L diff: inf, total time: 0.001338, loop time: 0.001336


/home/potzschf/repos/other_repos/pyTSEB/pyTSEB/TSEB.py:1975: RuntimeWarning: invalid value encountered in divide
  L_diff = np.asarray(np.fabs(L - L_old) / np.fabs(L_old), dtype=np.float32)


Iteration: 1, non-converged pixels: 1053694, max L diff: inf, total time: 3.533381, loop time: 3.532043
Iteration: 2, non-converged pixels: 1053694, max L diff: 0.610047, total time: 7.170734, loop time: 3.637353
Iteration: 3, non-converged pixels: 1053694, max L diff: 0.315542, total time: 10.836050, loop time: 3.665316
Iteration: 4, non-converged pixels: 1025793, max L diff: 0.463362, total time: 14.496434, loop time: 3.660384
Iteration: 5, non-converged pixels: 1025671, max L diff: 0.316671, total time: 16.617759, loop time: 2.121325
Iteration: 6, non-converged pixels: 1025147, max L diff: 0.112661, total time: 18.748591, loop time: 2.130831
Iteration: 7, non-converged pixels: 1023730, max L diff: 0.104790, total time: 20.191856, loop time: 1.443265
Iteration: 8, non-converged pixels: 1019948, max L diff: 0.088323, total time: 21.626554, loop time: 1.434698
Iteration: 9, non-converged pixels: 63579, max L diff: 0.022621, total time: 23.008242, loop time: 1.381688
Iteration: 10, non-

In [13]:
storPath = f"{tempOut}{LST_file.split('/')[-1].split('.')[0]}_ET.tif"
npTOdisk(et_daily_p, LST_file, storPath)

TypeError: unsupported operand type(s) for -: 'str' and 'str'